# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Write functions to parse specific information from a web page
* Iterate over successive web pages in order to create a dataset

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [5]:
from bs4 import BeautifulSoup
import requests
import re

In [10]:
html_page=requests.get("http://books.toscrape.com")
soup = BeautifulSoup(html_page.content, 'html.parser')

In [15]:
warning = soup.find('div', class_="alert alert-warning")
books=warning.nextSibling.nextSibling
titles=books.findAll('h3')
titles[0].find('a').attrs['title']

'A Light in the Attic'

In [17]:
def retrieve_titles(soup):
    all_the_titles=[]
    warning = soup.find('div', class_="alert alert-warning")
    books=warning.nextSibling.nextSibling
    titles=books.findAll('h3')
    for i in range(len(titles)):
        book_title=titles[i].find('a').attrs['title']
        all_the_titles.append(book_title)
    return all_the_titles
retrieve_titles(soup)

['A Light in the Attic',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History of Humankind',
 'The Requiem Red',
 'The Dirty Little Secrets of Getting Your Dream Job',
 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
 'The Black Maria',
 'Starving Hearts (Triangular Trade Trilogy, #1)',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
 'Rip it Up and Start Again',
 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
 'Olio',
 'Mesaerion: The Best Science Fiction Stories 1800-1849',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given html page and return a list of the star-ratings for the books. These star ratings should be formatted as integers.

In [72]:
ratings=books.findAll('p',class_=('star-rating'))
ratings[0].attrs['class'][1]

'Three'

In [78]:
def retrieve_ratings(soup):
    all_the_ratings=[]
    warning = soup.find('div', class_="alert alert-warning")
    books=warning.nextSibling.nextSibling
    ratings=books.findAll('p',class_=('star-rating'))
    number_ratings={'One':1,'Two':2,'Three':3,'Four':4,'Five':5}
    for i in range(len(ratings)):
        this_book=ratings[i].attrs['class'][1]
        int_rating=number_ratings[this_book]
        all_the_ratings.append(int_rating)
    return all_the_ratings
retrieve_ratings(soup)

[3, 1, 1, 4, 5, 1, 4, 3, 4, 1, 2, 4, 5, 5, 5, 3, 1, 1, 2, 2]

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [37]:
book_prices=books.findAll('p',class_='price_color')
float(book_prices[0].string[1:])

51.77

In [39]:
def retrieve_prices(soup):
    all_the_prices=[]
    warning = soup.find('div', class_="alert alert-warning")
    books=warning.nextSibling.nextSibling
    book_prices=books.findAll('p',class_='price_color')
    for i in range(len(book_prices)):
        this_book_price=float(book_prices[i].string[1:])
        all_the_prices.append(this_book_price)
    return all_the_prices
retrieve_prices(soup)

[51.77,
 53.74,
 50.1,
 47.82,
 54.23,
 22.65,
 33.34,
 17.93,
 22.6,
 52.15,
 13.99,
 20.66,
 17.46,
 52.29,
 35.02,
 57.25,
 23.88,
 37.59,
 51.33,
 45.17]

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [97]:
availabilities=books.findAll('p',class_='instock availability')
availabilities[0].text.strip()

'In stock'

In [100]:
def retrieve_availabilities(soup):
    all_books=[]
    warning = soup.find('div', class_="alert alert-warning")
    books=warning.nextSibling.nextSibling
    availabilities=books.findAll('p',class_='instock availability')
    for i in range(len(availabilities)):
        this_book=availabilities[i].text.strip()
        all_books.append(this_book)
    return all_books
retrieve_availabilities(soup)    

['In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock']

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [114]:
import pandas as pd

all_the_titles=[]
all_the_ratings=[]
all_the_prices=[]
all_the_avails=[]

for i in range(1,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_availabilities(soup)
    
    all_the_titles+=new_titles
    all_the_ratings+=new_star_ratings
    all_the_prices+=new_prices
    all_the_avails+=new_avails
    
df = pd.DataFrame([all_the_titles,all_the_ratings,all_the_prices,all_the_avails]).transpose()
df.columns=["Title","Rating","Price","Availability"]

In [125]:
df.head()

,Title,Rating,Price,Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used url hacking to generate each successive page url, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!